In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torchvision.datasets
from torchvision.transforms import v2
import torchvision.transforms.functional

In [ ]:
class Eye(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool1 = nn.AvgPool2d(2)
        #40x40
        self.conv2a = nn.Conv2d(16, 16, 3, padding=1)
        self.conv2b = nn.Conv2d(16, 32, 3, padding=1)
        self.batchNorm2 = nn.BatchNorm2d(32)
        #20x20
        self.conv3a = nn.Conv2d(32, 32, 3, padding=1)
        self.conv3b = nn.Conv2d(32, 64, 3, padding=1)
        self.batchNorm3 = nn.BatchNorm2d(64)
        #10x10

        self.conv4a = nn.Conv2d(64, 64, 3, padding=1)
        self.conv4b = nn.Conv2d(64, 64, 3, padding=1)
        self.batchNorm4 = nn.BatchNorm2d(64)
        # 5x5

        self.lin1 = nn.Linear(5*5*64, 64)
        self.lin2 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        #40x40
        x = self.conv2a(x)
        x = F.relu(x)
        x = self.conv2b(x)
        x = F.relu(x)
        x = self.batchNorm2(x)
        x = self.pool1(x)
        #20x20
        x = self.conv3a(x)
        x = F.relu(x)
        x = self.conv3b(x)
        x = F.relu(x)
        x = self.batchNorm3(x)
        x = self.pool1(x)
        #10x10
        x = self.conv4a(x)
        x = F.relu(x)
        x = self.conv4b(x)
        x = F.relu(x)
        x = self.batchNorm4(x)
        x = self.pool1(x)
        #5x5x128

        x = torch.flatten(x, 1)

        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return torch.clamp(tensor + torch.randn(tensor.size(), dtype=torch.float) * self.std + self.mean, 0, 1)
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:
to_torch = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True)
])
transforms = v2.Compose([
    v2.RandomHorizontalFlip(),
    v2.RandomAffine(15, [0.05,0.05], [0.9,1.1]),
    v2.GaussianBlur(3),
    v2.RandomApply([AddGaussianNoise(0, 0.15)], p=0.3),
    v2.RandomPosterize(bits=4),
    v2.RandomSolarize(threshold=0.9),
    v2.RandomAutocontrast(),
    v2.RandomAdjustSharpness(sharpness_factor=2)
])

In [ ]:
target_train = torchvision.datasets.ImageFolder("training", transform=to_torch)
target_valid = torchvision.datasets.ImageFolder("validation", transform=to_torch)

target_count = len([x[1] for x in target_train if x[1] == 0])
class_weights = [1/target_count, 1/(len(target_train)-target_count)]
class_weights = [class_weights[x[1]] for x in target_train]

sampler = torch.utils.data.WeightedRandomSampler(class_weights, len(target_train), replacement=True)

loader = torch.utils.data.DataLoader(target_train, batch_size=30, sampler=sampler)
val_loader = torch.utils.data.DataLoader(target_valid, batch_size=len(target_valid))

In [ ]:
x = next(iter(loader))

In [ ]:
display(torchvision.transforms.functional.to_pil_image(transforms(x[0][2])))

In [ ]:
model = Eye()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
lfunc = torch.nn.BCELoss()

In [ ]:
n_epochs = 85

for i in range(n_epochs):
    for batch in iter(loader):
        inputs, labels = batch
        inputs = transforms(inputs)
        optimizer.zero_grad()
        result = model(inputs)
        loss:torch.Tensor = lfunc(result, labels.reshape([len(labels),1]).float())
        loss.backward()
        optimizer.step()
    
    with torch.no_grad():
        for batch in val_loader:
            inputs, labels = batch
            result = model(inputs)
            loss:torch.Tensor = lfunc(result, labels.reshape([len(labels),1]).float())
            print(i, loss.item())


In [ ]:
with torch.no_grad():
    for batch in val_loader:
        inputs, labels = batch
        inputs = transforms(inputs)
        result = model(inputs)
        print((result < 0.5) == (labels.reshape([len(labels),1]) < 0.5))

In [ ]:
with open("visual_model.pth", "wb") as f:
    torch.save(model, f)